# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import geopandas as gpd
import pandas as pd
import requests
import matplotlib.pyplot as plt
import holoviews as hv


# Import API key
from api_keys import geoapify_key

In [17]:
#!pip install --upgrade hvplot holoviews

  Obtaining dependency information for hvplot from https://files.pythonhosted.org/packages/bd/bc/584a7c21acb408af9c3aaab3b087c21ac624605fbd5a4432455529ef851a/hvplot-0.9.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for holoviews from https://files.pythonhosted.org/packages/dc/25/8f22bc587eb77f399c5d3218b9743298ee7f0eedd937c6169d5a6b4db09e/holoviews-1.18.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
    --------------------------------------- 0.0/3.2 MB 2.0 MB/s eta 0:00:02
   -- ------------------------------------- 0.2/3.2 MB 3.0 MB/s eta 0:00:02
   ------- -------------------------------- 0.6/3.2 MB 5.2 MB/s eta 0:00:01
   ----------------- ---------------------- 1.4/3.2 MB 8.1 MB/s eta 0:00:01
   ---------------------------- ----------- 2.3/3.2 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------  3.2/3.2 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 11.4 MB/s

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kachug,53.9606,105.8817,239.73,100.0,81.0,0.73,RU,1.706384e+09
1,1,wailua homesteads,22.0669,-159.3780,296.90,72.0,40.0,2.06,US,1.706384e+09
2,2,pekanbaru,0.5333,101.4500,298.33,100.0,100.0,0.51,ID,1.706384e+09
3,3,grytviken,-54.2811,-36.5092,278.24,95.0,100.0,1.75,GS,1.706384e+09
4,4,kalajoki,64.2500,23.9500,271.99,77.0,100.0,7.62,FI,1.706384e+09


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
#%pdb   #%debug

In [19]:
# Create a scatter plot with hvplot (code sourced from https://holoviz.org/tutorial/Composing_Plots.html, 1/27/24)

from holoviews.element.tiles import EsriImagery
EsriImagery()

scatter_plot = city_data_df.hvplot.points(
    x='Lng', y='Lat', c='Humidity', hover_cols=['City', 'Humidity'], title='City Humidity Map', tiles='EsriImagery', line_color='black')


# # Display the plot
scatter_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

In [20]:
%%capture --no-display

# Configure the map plot. c is color, but we can also set to a variable. geo=True indicates plot is geospatial
humidity_scatter_plot = city_data_df.hvplot.points(
    x='Lng', y='Lat', c='Humidity', hover_cols=['City', 'Humidity'], title='City Humidity Map', tiles='EsriImagery', line_color='black')


#Display the map
humidity_scatter_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [21]:

# Narrow down cities that fit criteria and drop any results with null values
city_data_locations = city_data_df[(city_data_df['Max Temp'] > 65) & (city_data_df['Humidity'] < 65) & (city_data_df['Wind Speed'] < 15) 
            & (city_data_df['Cloudiness'] < 45)]

# Drop any rows with null values
city_data_locations.dropna(axis=0)

# Display sample data
city_data_locations.count()

City_ID       98
City          98
Lat           98
Lng           98
Max Temp      98
Humidity      98
Cloudiness    98
Wind Speed    98
Country       97
Date          98
dtype: int64

### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ('')

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,kachug,53.9606,105.8817,239.73,100.0,81.0,0.73,RU,1.706384e+09,
1,1,wailua homesteads,22.0669,-159.3780,296.90,72.0,40.0,2.06,US,1.706384e+09,
2,2,pekanbaru,0.5333,101.4500,298.33,100.0,100.0,0.51,ID,1.706384e+09,
3,3,grytviken,-54.2811,-36.5092,278.24,95.0,100.0,1.75,GS,1.706384e+09,
4,4,kalajoki,64.2500,23.9500,271.99,77.0,100.0,7.62,FI,1.706384e+09,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [5]:

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    
# Set parameters to search for a hotel
#ChatGPT 2024, personal communication, January 27,2024
    radius = 10000
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    params = {
    "apiKey":geoapify_key,
    "filters":filters,
    "bias":bias,
    "type":"accommodation.hotel"
        
    }
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #ChatGPT 2024, personal communication, January 27,2024
   
    params["filters"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
      
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
      
        
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    #print to debug
    #print("API Response:", name_address) #  -- this is no good I get this output
   # API Response: {'statusCode': 400, 'error': 'Bad Request', 'message': '"filter[0]" does not match any of the allowed types'}
    #tiksi - nearest hotel: No hotel found
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
kachug - nearest hotel: Империя
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
pekanbaru - nearest hotel: Hotel Radja
grytviken - nearest hotel: Shorty's Motel
kalajoki - nearest hotel: Hotel Rantakalla
pacific grove - nearest hotel: Pacific Grove Inn
severo-yeniseyskiy - nearest hotel: Актолик
bilibino - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
waitangi - nearest hotel: Hotel Chathams
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
namibe - nearest hotel: Hotel Chik Chik Namibe
hibaiyo - nearest hotel: RHR Tourist Inn
mount isa - nearest hotel: Ibis Styles
iqaluit - nearest hotel: Frobisher Inn
puerto natales - nearest hotel: Dorotea Patagonia Hostel
ribeira grande - nearest hotel: Hospedaria JSF
port-aux-francais - nearest hotel: Keravel
edinburgh of the seven seas - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
titusville - nearest hotel: Palace Hotel
min

hell-ville - nearest hotel: ABUD HOTEL
vingt cinq - nearest hotel: No hotel found
ondjiva - nearest hotel: Águia Verde Hotel
noumea - nearest hotel: Hotel Gondwana
newman - nearest hotel: Hampton Inn & Suites
rawson - nearest hotel: Hotel Deportivo
khash - nearest hotel: هتل قصِر
tortoli - nearest hotel: La Corte
rimouski - nearest hotel: Hôtel St-Germain Rimouski
lebanon - nearest hotel: Ramada
tranovaho - nearest hotel: No hotel found
montemor-o-novo - nearest hotel: Montemor
brisas barra de suchiate - nearest hotel: Mazari II
tofol - nearest hotel: Nautilus Resort
saldanha - nearest hotel: Hoedjesbaai
arteche - nearest hotel: RJV hôtel
arafat - nearest hotel: ابراج فجر الايمان
la passe - nearest hotel: L'Estuaire
cidreira - nearest hotel: Hotel Castelo
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
moengo - nearest hotel: Amazonie accueil
'ohonua - nearest hotel: No hotel found
anna regina - nearest hotel: Jaigobin Hotel
mostys'ka - nearest hotel: No hotel found
al kha

KeyboardInterrupt: 

In [23]:
hotel_df.describe()

,City_ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,608.00000,579.000000,579.000000,579.000000,579.000000,579.000000,579.000000,5.790000e+02
mean,303.50000,19.045323,16.573519,285.526667,73.191710,62.129534,3.652401,1.706384e+09
std,175.65876,32.444612,90.085043,17.172883,22.369256,39.566625,2.707307,8.059343e+01
min,0.00000,-54.800000,-176.559700,229.040000,11.000000,0.000000,0.000000,1.706384e+09
25%,151.75000,-7.157200,-63.146700,277.040000,63.000000,20.000000,1.665000,1.706384e+09
50%,303.50000,22.066900,20.225100,290.920000,79.000000,75.000000,3.090000,1.706384e+09
75%,455.25000,46.417600,93.320400,298.325000,90.000000,100.000000,5.130000,1.706384e+09
max,607.00000,78.218600,178.004200,311.180000,100.000000,100.000000,16.770000,1.706384e+09


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
from holoviews.element.tiles import EsriImagery
EsriImagery()

scatter_plot = hotel_df.hvplot.points(
    x='Lng', y='Lat', c='Humidity', hover_cols=['City', 'Humidity', 'Country', 'Hotel Name'], title='City Humidity Map', tiles='EsriImagery', line_color='black')


# Display the map
scatter_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)